# Resume Specialist Using Self-Refine Technique

You want to write a resume, but you don't like writing like me! Or you are not in the mood. Are you looking for a resume specialist who writes a resume for you while you talk about your skills, education, experience, etc. If so, please read this post on how to create a resume specialist that generates and criticizes itself until it comes up with the most optimal resume for the user.

This resume specialist uses a self reflection technique to criticize itself. For that purpose, I used LangGraph. You can read more about LangGraph here. In self reflection technique, LLM observes its past actions and evaluates them in order to improve the quality of output later on. \
Here are the overview of the use case:
* Generate the transcription of the audio resume using a speech-to-text model.
* Generate the first draft of the resume.
* Criticize the generated resume.
* Continue until the stop condition is satisfied.

<center><figure><img src="imgs/resume_specialist diagram.jpg" alt="drawing" width="1000"/><figcaption>Fig. 1: Resume specialist architecture</figcaption></figure></center>   

In [2]:
! pip install -U --quiet  langchain langgraph langchain-fireworks openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.12.1 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.2 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.2 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.1.0 which is incompatible.
sagema

In [3]:
import os
from getpass import getpass
os.environ['FIREWORKS_API_KEY'] = getpass('Enter your FIREWORKS API key: ')
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your FIREWORKS API key:  ········
Enter your OpenAI API Key:  ········


In [4]:
from langchain.prompts import ChatMessagePromptTemplate
from typing import List, Sequence
from langchain_fireworks import ChatFireworks
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from typing import List, Sequence
from langgraph.graph import END, MessageGraph

As an LLM model, I choose Mixtral 8x7B instruction model (aka chat model). Mixtral-8x7B Large Language Model (LLM) is a pretrained generative Sparse Mixture of Experts. The Mistral-8x7B outperforms Llama 2 70B on most benchmarks with 6x faster inference. Read more about Mixtral [here](https://mistral.ai/news/mixtral-of-experts/)

In [5]:
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct", max_tokens = 32768)
# llm = ChatMistralAI(model="mistral-small-latest", model_kwargs={"max_tokens": 32768})

In [6]:
async def generate(state: Sequence[BaseMessage]):
    prompt = ChatPromptTemplate.from_messages(
        [
            "system",
            "You are a resume assistant tasked with writing excellent resumes."
            " Generate the best resume possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    
    chain = prompt | llm
    # print("###### message type: #######",messages[0].type) 
    return await chain.ainvoke({"messages": state})   

In [7]:
async def refine(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    review_prompt = ChatPromptTemplate.from_messages(
        [
            "system",
            "You are a resume reviewer evaluating a resume submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations.",
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    switch_roles = {"ai": HumanMessage, "human": AIMessage}
    transformed_messages = [messages[0]] + [
    switch_roles[msg.type](content=msg.content) for msg in messages[1:]]
    
    chain = review_prompt | llm
    refinement = await chain.ainvoke({"messages": transformed_messages})
    return HumanMessage(refinement.content)

In [16]:
def critisize(state: List[BaseMessage]):
    print("$$$$$$$ messages length: $$$$$$$$$", len(state))
    if len(state) >= 6:
        # End after 3 iterations
        return END
    return "refine"

## We define our graph as follows:
* generate node: is responsible for generating a resume based on the resume transcription of the user
* reflect node: criticizes the generated resume and gives several recommendations on it
* should_continue : is a conditional edge which decides to repeat the process of generat and review or quit the loop and output the final version
* reflect - generate: is a normal edge from reflect node to generate node which causes the LLM to revise the previous attemp and apply the new comments to the CV.

<center><figure><img src="imgs/resume_specialist.jpg" alt="drawing" width="600"/><figcaption>Fig. 1: Graph of the example</figcaption></figure></center>   

In [9]:
graph = MessageGraph()

In [10]:
graph.add_node("generate",generate)
graph.add_node("refine",refine)
graph.set_entry_point("generate")


graph.add_conditional_edges("generate", critisize)
graph.add_edge("refine", "generate")
app = graph.compile()

In [11]:
from openai import OpenAI
client = OpenAI()

audio_file = open("docs/resume.m4a", "rb")
resume_transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(resume_transcription)

My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation and I am able to work with machine learning libraries such as Scikit-learn. If you want to reach me, my email is joe.smith at example.com



In [17]:
from os import linesep
input = HumanMessage(content = f"""Write a resume based on the following information: \n
                                Resume : {resume_transcription} \n
                                """)
outputs = []
async for event in app.astream(input):
    outputs.append(event)
    print(event)

    print("\n---\n")

$$$$$$$ messages length: $$$$$$$$$ 2
{'generate': AIMessage(content='**Joe Smith**\nPhD in Computer Science, University of Toronto (2017)\nEmail: joe.smith@example.com\n\n**Professional Summary**\nExperienced Machine Learning Engineer and Senior Language Model Researcher with a strong background in developing machine learning models and conducting research on large language models. Proficient in Python and deep learning frameworks such as PyTorch and TensorFlow. Skilled in using Panda and SQL for data manipulation and machine learning libraries such as Scikit-learn.\n\n**Professional Experience**\n\n*Senior Language Model Researcher, Y Company (2022-Present)*\n\n* Conduct research on large language models (LLM) and their fine-tuning techniques\n* Prototype use cases using available large language tools\n\n*Machine Learning Engineer, X Company (2017-2022)*\n\n* Developed various machine learning models for X project\n* Mentored junior developers\n* Deployed and monitored models in produ

In [15]:
print(event)

{'generate': AIMessage(content='Joe Smith\n------------------\nPhD in Computer Science - University of Toronto (2017)\n\nContact Information\n------------------\nEmail: joe.smith@example.com\n\nSummary Statement\n------------------\nExperienced machine learning engineer and language model researcher with a PhD in computer science. Proficient in Python and deep learning frameworks such as PyTorch and TensorFlow. Skilled in data manipulation using Panda and SQL, and experienced in mentoring junior developers.\n\nWork Experience\n------------------\nSenior Language Model Researcher - Y Company (2022-Present)\n\n* Conduct research on large language models (LLM) and develop fine-tuning and augmentation techniques\n* Prototype use cases using available large language tools, resulting in a 30% increase in efficiency\n\nMachine Learning Engineer - X Company (2017-2022)\n\n* Developed and deployed 5 machine learning models, resulting in a 20% increase in efficiency\n* Mentored junior developers